# Calibration

This notebook will show you how to quickly calibrate standard VAMPIRES imaging data

## Setup and Imports

In [1]:
from astropy.io import fits
import numpy as np
from pathlib import Path
import proplot as pro
import tqdm.auto as tqdm
import os

pro.rc["style"] = "ggplot"
pro.rc["image.origin"] = "lower"


In [2]:
datadir = Path("data")


In [3]:
# get filelist
filelist = list(datadir.glob("VMPA*[0123456789].fits"))
dark_files = list(datadir.glob("darks*cam[12].fits"))
print(f"Total files: {len(filelist)}")


Total files: 16


## Step 1: Fixing FITS headers

There are some unfortunate bugs with some of the FITS headers from VAMPIRES. There are some functions that try and fix these bugs, and ignore files that don't have any problems.

In [4]:
from vampires_dpp.fixes import fix_header

# skip=True will load file from disk instead of reprocessing
iter = tqdm.tqdm(filelist, desc="Fixing FITS headers")
fixed_files = [fix_header(f, skip=True) for f in iter]


Fixing FITS headers:   0%|          | 0/16 [00:00<?, ?it/s]

In [5]:
from vampires_dpp.headers import observation_table

# create a dataframe from the FITS headers for easily sorting through data
# here I sort the values in perfect order for differential image analysis
table = observation_table(fixed_files).sort_values(
    ["DATE", "U_PLSTIT", "U_FLCSTT", "U_CAMERA"]
)
table[["OBJECT", "DATE", "U_HWPANG", "U_FLCSTT", "U_CAMERA"]]


,OBJECT,DATE,U_HWPANG,U_FLCSTT,U_CAMERA
11,ABAUR,2022-02-24T05:44:51,0.0,1,1.0
10,ABAUR,2022-02-24T05:44:51,0.0,2,1.0
9,ABAUR,2022-02-24T05:44:52,0.0,1,2.0
8,ABAUR,2022-02-24T05:44:52,0.0,2,2.0
6,ABAUR,2022-02-24T05:46:57,45.0,1,1.0
4,ABAUR,2022-02-24T05:46:57,45.0,2,1.0
1,ABAUR,2022-02-24T05:46:58,45.0,1,2.0
14,ABAUR,2022-02-24T05:46:58,45.0,2,2.0
13,ABAUR,2022-02-24T05:48:19,22.5,1,1.0
5,ABAUR,2022-02-24T05:48:19,22.5,2,1.0


## Step 2: Basic Image Calibration

For typical VAMPIRES data we only do dark subtraction. The following calibration routines will dark subtract, as well as discard leading garbage frames, and flip the cameras orientation to remove the flip from the beamsplitter cube.


In [6]:
from vampires_dpp.calibration import make_dark_file

# make master darks
master_darks = (
    make_dark_file(dark_files[0], output=datadir / "master_dark_cam1.fits"),
    make_dark_file(dark_files[1], output=datadir / "master_dark_cam2.fits"),
)


In [7]:
from vampires_dpp.calibration import calibrate_file

# calibrate data
calibrated_files = []
iter = tqdm.tqdm(table["path"], desc="Calibrating images")
for filename, cam in zip(iter, table["U_CAMERA"]):
    dark_file = master_darks[int(cam - 1)]
    f = calibrate_file(filename, dark=dark_file, skip=True)
    calibrated_files.append(f)


Calibrating images:   0%|          | 0/16 [00:00<?, ?it/s]

## Step 3: Frame selection and image registration (lucky imaging)

In [8]:
from vampires_dpp.satellite_spots import lamd_to_pixel

# satellite spot radius in pixels
radius = lamd_to_pixel(31.6, "750-50")
radius


100.56605913350828

In [9]:
from vampires_dpp.frame_selection import frame_select_file

iter = tqdm.tqdm(
    calibrated_files, desc="Removing worst frames measured from satellite spots"
)
selected_files = [
    frame_select_file(
        f, metric="l2norm", q=0.4, coronagraphic=True, radius=radius, skip=True
    )
    for f in iter
]


Removing worst frames measured from satellite spots:   0%|          | 0/16 [00:00<?, ?it/s]

In [10]:
from vampires_dpp.image_registration import measure_offsets, register_file
from vampires_dpp.image_processing import collapse_file

iter = tqdm.tqdm(
    selected_files, desc="Measuring frame offsets, registering, and collapsing frames"
)
collapsed_files = []
for filename in iter:
    offset_file = measure_offsets(
        filename,
        method="moffat",
        coronagraphic=True,
        background_subtract=True,
        radius=radius,
        skip=True,
    )
    registered_file = register_file(filename, offset_file, skip=True)
    f = collapse_file(registered_file, skip=True)
    collapsed_files.append(f)


Measuring frame offsets, registering, and collapsing frames:   0%|          | 0/16 [00:00<?, ?it/s]